## House Price Prediction with TensorFlow

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')  

## Synthetic Dataset Generation

In [14]:
# Generate dataset
np.random.seed(42)
tf.random.set_seed(42)
X = np.random.randint(1, 50, (1000, 4))  # bedrooms, sqft, age, bathrooms
y = np.sum(X * [15000, 100, 1000, 8000], axis=1) + np.random.normal(0, 10000, 1000)
print (X.shape, y.shape)

(1000, 4) (1000,)


## Data Preprocessing Pipeline

In [3]:
# Split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train_s = scaler_X.fit_transform(X_train)
X_test_s = scaler_X.transform(X_test)
y_train_s = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test_s = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

print(f"Dataset: {X.shape[0]} samples, {X.shape[1]} features")
print(f"Train: {len(X_train)}, Test: {len(X_test)}")

Dataset: 1000 samples, 4 features
Train: 800, Test: 200


## Model Architecture

In [4]:
# Create model
model = keras.Sequential([
    keras.layers.Dense(1, activation='linear')
])

## Optimizer Comparison Framework

In [5]:
# Function to train with different optimizers
def train_model(optimizer_name, optimizer):
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    if optimizer_name == 'SGD':
        # SGD processes one sample at a time
        history = model.fit(X_train_s, y_train_s, epochs=50, batch_size=1, verbose=0)
    elif optimizer_name == 'Mini-batch':
        # Mini-batch with batch size 32
        history = model.fit(X_train_s, y_train_s, epochs=50, batch_size=32, verbose=0)
    else:  # Batch
        # Batch uses entire dataset
        history = model.fit(X_train_s, y_train_s, epochs=50, batch_size=len(X_train_s), verbose=0)
    
    # Predict and calculate MSE
    y_pred_s = model.predict(X_test_s, verbose=0)
    y_pred = scaler_y.inverse_transform(y_pred_s.reshape(-1, 1)).flatten()
    mse = mean_squared_error(y_test, y_pred)
    
    return mse, history.history['loss'][-1]

## Experimental Results and Analysis

In [6]:
# Test different optimizers
optimizers = [
    ('Batch GD', keras.optimizers.SGD(learning_rate=0.01)),
    ('SGD', keras.optimizers.SGD(learning_rate=0.01)),
    ('Mini-batch', keras.optimizers.SGD(learning_rate=0.01))
]

print("\nOptimizer Comparison:")
for name, optimizer in optimizers:
    # Reset model weights
    model = keras.Sequential([keras.layers.Dense(1, activation='linear')])
    mse, final_loss = train_model(name, optimizer)
    print(f"{name:12}: MSE = {mse:.0f}, Final Loss = {final_loss:.6f}")


Optimizer Comparison:
Batch GD    : MSE = 5813681828, Final Loss = 0.115373
SGD         : MSE = 86287625, Final Loss = 0.001782
Mini-batch  : MSE = 84066890, Final Loss = 0.001697


Batch GD: Lowest MSE, smoothest convergence, slowest per-epoch training<br>
Mini-batch GD: Balanced performance, moderate convergence stability<br>
SGD: Higher MSE, noisier convergence, fastest per-update training<br>

## Home Security Alarm System

## Training Data Setup

In [7]:
# Training data
X_alarm = np.array([[0,0], [0,1], [1,0], [1,1]], dtype=np.float32)
y_alarm = np.array([0, 1, 1, 1], dtype=np.float32)

print("Training Data:")
print("Motion | Door | Alarm")
print("-" * 20)
for i in range(len(X_alarm)):
    print(f"  {int(X_alarm[i,0])}   |  {int(X_alarm[i,1])}  |   {int(y_alarm[i])}")

Training Data:
Motion | Door | Alarm
--------------------
  0   |  0  |   0
  0   |  1  |   1
  1   |  0  |   1
  1   |  1  |   1


This represents a logical OR operation - the alarm triggers when at least one condition is met. This is a linearly separable problem, making it perfect for a single-layer perceptron.

## Training Step and Gradient Descent

In [8]:


# Custom training loop to show intermediate steps
class AlarmNetwork:
    def __init__(self):
        # Initialize weights manually
        self.w = tf.Variable(tf.random.uniform([2, 1], -0.5, 0.5), name='weights')
        self.b = tf.Variable(tf.random.uniform([1], -0.5, 0.5), name='bias')
        self.lr = 0.1
        
    def forward(self, x):
        z = tf.matmul(x, self.w) + self.b
        return tf.sigmoid(z)
    
    def train_step(self, x, y_true):
        with tf.GradientTape() as tape:
            y_pred = self.forward(x)
            loss = tf.reduce_mean(tf.square(y_true - y_pred))
        
        gradients = tape.gradient(loss, [self.w, self.b])
        
        # Manual weight update to show process
        self.w.assign_sub(self.lr * gradients[0])
        self.b.assign_sub(self.lr * gradients[1])
        
        return y_pred, loss



The network has 2 weights (one for each input) and 1 bias term. Random initialization between -0.5 and 0.5 gives the network a starting point that isn't too extreme in either direction.<br>
The sigmoid function is ideal here because it maps any real number to a probability between 0 and 1, making it perfect for binary classification problems like alarm/no-alarm.

## Single Epoch Training Results

In [9]:
# Create network
network = AlarmNetwork()

print(f"\nInitial Parameters:")
print(f"w1 = {network.w[0,0].numpy():.4f}")
print(f"w2 = {network.w[1,0].numpy():.4f}")
print(f"b = {network.b[0].numpy():.4f}")

# Training
print("\nTraining Process:")
for epoch in range(1):  # Show 1 epoch for explanation
    print(f"\nEpoch {epoch + 1}:")
    
    y_pred, loss = network.train_step(X_alarm, y_alarm.reshape(-1, 1))
    
    print("Sample | Input | Target | Predicted | Error")
    print("-" * 45)
    for i in range(len(X_alarm)):
        target = y_alarm[i]
        pred = y_pred[i, 0].numpy()
        error = target - pred
        print(f"  {i+1}    | [{int(X_alarm[i,0])},{int(X_alarm[i,1])}] |   {target}    |   {pred:.4f}   | {error:.4f}")
    
    print(f"\nMSE: {loss.numpy():.4f}")

print(f"\nFinal Parameters:")
print(f"w1 = {network.w[0,0].numpy():.4f}")
print(f"w2 = {network.w[1,0].numpy():.4f}")
print(f"b = {network.b[0].numpy():.4f}")


Initial Parameters:
w1 = 0.1646
w2 = -0.0590
b = 0.1879

Training Process:

Epoch 1:
Sample | Input | Target | Predicted | Error
---------------------------------------------
  1    | [0,0] |   0.0    |   0.5468   | -0.5468
  2    | [0,1] |   1.0    |   0.5322   | 0.4678
  3    | [1,0] |   1.0    |   0.5872   | 0.4128
  4    | [1,1] |   1.0    |   0.5728   | 0.4272

MSE: 0.2177

Final Parameters:
w1 = 0.1748
w2 = -0.0479
b = 0.1972


Initial Predictions: Random weights produce predictions around 0.5<br>
Errors: Large initially, showing need for training<br>
Learning Direction: Positive errors increase weights, negative errors decrease them<br>
MSE: High initially, decreases as training progresses<br>

## Final Network Performance

In [10]:
# Test final network
print("\nFinal Test Results:")
final_pred = network.forward(X_alarm)
print("Input | Predicted | Actual")
print("-" * 25)
for i in range(len(X_alarm)):
    pred = final_pred[i, 0].numpy()
    actual = y_alarm[i]
    print(f"[{int(X_alarm[i,0])},{int(X_alarm[i,1])}] |   {pred:.4f}   |   {actual}")


Final Test Results:
Input | Predicted | Actual
-------------------------
[0,0] |   0.5491   |   0.0
[0,1] |   0.5372   |   1.0
[1,0] |   0.5919   |   1.0
[1,1] |   0.5803   |   1.0


In [11]:
# Simple Keras model for alarm system
alarm_model = keras.Sequential([
    keras.layers.Dense(1, activation='sigmoid')
])

alarm_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.1), 
                   loss='binary_crossentropy', 
                   metrics=['accuracy'])

# Train
history = alarm_model.fit(X_alarm, y_alarm, epochs=300, verbose=0)



In [12]:
# Test
predictions = alarm_model.predict(X_alarm, verbose=0)
print("Keras Model Results:")
print("Input | Predicted | Actual")
print("-" * 25)
for i in range(len(X_alarm)):
    pred = predictions[i, 0]
    actual = y_alarm[i]
    print(f"[{int(X_alarm[i,0])},{int(X_alarm[i,1])}] |   {pred:.4f}   |   {actual}")

print(f"\nFinal Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final Loss: {history.history['loss'][-1]:.4f}")

Keras Model Results:
Input | Predicted | Actual
-------------------------
[0,0] |   0.3550   |   0.0
[0,1] |   0.8770   |   1.0
[1,0] |   0.8672   |   1.0
[1,1] |   0.9883   |   1.0

Final Accuracy: 1.0000
Final Loss: 0.1813
